In [7]:
// Read json files, how to flatten, array/struct

// Whereever there is an Array use explode and wherever there is a Struct use . to reach to the desired key
// There are tail recursive methods written manually by programmers but use those only entire df to be flattened, check the last cell

val metadata_read_path = spark.read.format("json"
                                          ).option("multiline","true"
                                          ).load("json_read_spark_scala.json")

metadata_read_path.printSchema()


root
 |-- @context: struct (nullable = true)
 |    |-- @language: string (nullable = true)
 |    |-- @vocab: string (nullable = true)
 |    |-- column: string (nullable = true)
 |    |-- data: struct (nullable = true)
 |    |    |-- @id: string (nullable = true)
 |    |    |-- @type: string (nullable = true)
 |    |-- dataType: struct (nullable = true)
 |    |    |-- @id: string (nullable = true)
 |    |    |-- @type: string (nullable = true)
 |    |-- extract: string (nullable = true)
 |    |-- field: string (nullable = true)
 |    |-- fileProperty: string (nullable = true)
 |    |-- format: string (nullable = true)
 |    |-- includes: string (nullable = true)
 |    |-- isEnumeration: string (nullable = true)
 |    |-- jsonPath: string (nullable = true)
 |    |-- ml: string (nullable = true)
 |    |-- parentField: string (nullable = true)
 |    |-- path: string (nullable = true)
 |    |-- recordSet: string (nullable = true)
 |    |-- references: string (nullable = true)
 |    |-- rege

metadata_read_path: org.apache.spark.sql.DataFrame = [@context: struct<@language: string, @vocab: string ... 23 more fields>, @type: string ... 7 more fields]


In [17]:
metadata_read_path.select("@context.transform").show(false)

+------------+
|transform   |
+------------+
|ml:transform|
+------------+



In [34]:
metadata_read_path.withColumn("recordSet_exploded", explode(array("recordSet"))).printSchema()

root
 |-- @context: struct (nullable = true)
 |    |-- @language: string (nullable = true)
 |    |-- @vocab: string (nullable = true)
 |    |-- column: string (nullable = true)
 |    |-- data: struct (nullable = true)
 |    |    |-- @id: string (nullable = true)
 |    |    |-- @type: string (nullable = true)
 |    |-- dataType: struct (nullable = true)
 |    |    |-- @id: string (nullable = true)
 |    |    |-- @type: string (nullable = true)
 |    |-- extract: string (nullable = true)
 |    |-- field: string (nullable = true)
 |    |-- fileProperty: string (nullable = true)
 |    |-- format: string (nullable = true)
 |    |-- includes: string (nullable = true)
 |    |-- isEnumeration: string (nullable = true)
 |    |-- jsonPath: string (nullable = true)
 |    |-- ml: string (nullable = true)
 |    |-- parentField: string (nullable = true)
 |    |-- path: string (nullable = true)
 |    |-- recordSet: string (nullable = true)
 |    |-- references: string (nullable = true)
 |    |-- rege

In [45]:
metadata_read_path.select("recordSet.name").printSchema()

root
 |-- name: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [46]:
 metadata_read_path.select("recordSet.@type").printSchema()

root
 |-- @type: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [49]:
metadata_read_path.select("@context.data.@id").show(false)

+-------+
|@id    |
+-------+
|ml:data|
+-------+



In [ ]:
// https://stackoverflow.com/questions/61863489/flatten-nested-json-in-scala-spark-dataframe


import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import scala.annotation.tailrec
import scala.util.Try

implicit class DFHelpers(df: DataFrame) {
    def columns = {
      val dfColumns = df.columns.map(_.toLowerCase)
      df.schema.fields.flatMap { data =>
        data match {
          case column if column.dataType.isInstanceOf[StructType] => {
            column.dataType.asInstanceOf[StructType].fields.map { field =>
              val columnName = column.name
              val fieldName = field.name
              col(s"${columnName}.${fieldName}").as(s"${columnName}_${fieldName}")
            }.toList
          }
          case column => List(col(s"${column.name}"))
        }
      }
    }

    def flatten: DataFrame = {
      val empty = df.schema.filter(_.dataType.isInstanceOf[StructType]).isEmpty
      empty match {
        case false =>
          df.select(columns: _*).flatten
        case _ => df
      }
    }
    def explodeColumns = {
      @tailrec
      def columns(cdf: DataFrame):DataFrame = cdf.schema.fields.filter(_.dataType.typeName == "array") match {
        case c if !c.isEmpty => columns(c.foldLeft(cdf)((dfa,field) => {
          dfa.withColumn(field.name,explode_outer(col(s"${field.name}"))).flatten
        }))
        case _ => cdf
      }
      columns(df.flatten)
    }
}

// Exiting paste mode, now interpreting.

import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import scala.annotation.tailrec
import scala.util.Try
defined class DFHelpers

